In [1]:
import pandas as pd
import numpy as np
import torch.nn.functional as F
import torch.nn as nn
import torch.nn.functional as F
import torch
import re
import nltk

from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from nltk.corpus import stopwords
from nltk.stem.wordnet import WordNetLemmatizer
from torchtext.data.utils import get_tokenizer
from torchtext.vocab import build_vocab_from_iterator
from torch.utils.data import Dataset

nltk.download('stopwords')
encoder = LabelEncoder()

STOPWORDS = set(stopwords.words('english'))

[nltk_data] Downloading package stopwords to
[nltk_data]     /common/home/ac1771/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
if torch.cuda.is_available():
  print('Good to go!')
else:
  print('Please set GPU via Edit -> Notebook Settings.')
device = torch.device('cuda:0')

Good to go!


This code uses the Huffington Post News Category dataset from kaggle which can be found here: [Source](https://www.kaggle.com/datasets/rmisra/news-category-dataset)

In [3]:
class MyDataset(Dataset):
 
    def __init__(self,file_name):

        self.df = pd.read_csv(file_name)#.drop(columns=['authors','link','date'])
        
        # change the dtype to category
        self.df = self.df.astype({'category': 'category'})
        self.df = self.df.astype({'short_description': 'str'})
        self.df['num_cat'] = self.df['category'].cat.codes

        self.sentances = self.df['short_description']
        self.labels = self.df['num_cat']

        self.x_train = None
        self.x_test = None
        self.y_train = None
        self.y_test = None

    def clean_data(self):

        for i in range(len(self.sentances)):
            text = self.sentances[i]
            whitespace = re.compile(r"\s+")
            user = re.compile(r"(?i)@[a-z0-9_]+")
            text = whitespace.sub(' ', text)
            text = user.sub('', text)
            text = re.sub(r"\[[^()]*\]","", text)
            text = re.sub("\d+", "", text)
            text = re.sub(r'[^\w\s]','',text)
            text = re.sub(r"(?:@\S*|#\S*|http(?=.*://)\S*)", "", text)
            text = text.lower()
        
            # removing stop-words
            text = [word for word in text.split() if word not in list(STOPWORDS)]
        
            # word lemmatization
            sentence = []
            for word in text:
                lemmatizer = WordNetLemmatizer()
                sentence.append(lemmatizer.lemmatize(word,'v'))
            
            self.sentances.iloc[i] = ' '.join(sentence)
    
    def get_train(self):
        return zip(self.y_train, self.x_train)
    
    def get_test(self):
        return zip(self.y_test, self.x_test)
    
    def split_test_train(self):
        self.x_train, self.x_test, self.y_train, self.y_test = train_test_split(self.sentances, self.labels, test_size=0.20)

    def __len__(self):
        return len(self.y_train)

    def __getitem__(self,idx):
        return self.x_train[idx],self.y_train[idx]


In [4]:
huffPo = MyDataset('/common/users/shared/cs543_fall22_group3/huffpo/lemmatized.csv')

In [41]:
# huffPo.clean_data()

In [5]:
huffPo.split_test_train()

In [6]:
tokenizer = get_tokenizer('basic_english')
train_iter = huffPo.get_train()

def yield_tokens(data_iter):
    for _, text in data_iter:
        yield tokenizer(text)

vocab = build_vocab_from_iterator(yield_tokens(train_iter), specials=["<unk>"])
vocab.set_default_index(vocab["<unk>"])

In [7]:
text_pipeline = lambda x: vocab(tokenizer(x))
label_pipeline = lambda x: int(x)

In [8]:
from torch.utils.data import DataLoader

def collate_batch(batch):
    label_list, text_list, offsets = [], [], [0]
    for (_label, _text) in batch:
         label_list.append(label_pipeline(_label))
         processed_text = torch.tensor(text_pipeline(_text), dtype=torch.int64)
         text_list.append(processed_text)
         offsets.append(processed_text.size(0))
    label_list = torch.tensor(label_list, dtype=torch.int64)
    offsets = torch.tensor(offsets[:-1]).cumsum(dim=0)
    text_list = torch.cat(text_list)
    return label_list.to(device), text_list.to(device), offsets.to(device)

train_iter = huffPo.get_train()
dataloader = DataLoader(train_iter, batch_size=8, shuffle=False, collate_fn=collate_batch)


In [80]:
from torch import nn

class TextClassificationModel(nn.Module):

    def __init__(self, vocab_size, embed_dim, num_class):
        super(TextClassificationModel, self).__init__()
        
        # Hyperparameters
        self.embed_dim = embed_dim
        self.LSTM_layers = 2
        self.vocab_size = vocab_size

        self.dropout = nn.Dropout(0.5)
        self.embedding = nn.EmbeddingBag(vocab_size, embed_dim, sparse=True)
        self.lstm = nn.LSTM(input_size=embed_dim, hidden_size=embed_dim, num_layers=self.LSTM_layers, batch_first=True)
        self.fc = nn.Linear(embed_dim, num_class)

    def forward(self, text, offsets):

        # Hidden and cell state definion
        h = torch.zeros((self.LSTM_layers, self.embed_dim)).to(device)
        c = torch.zeros((self.LSTM_layers, self.embed_dim)).to(device)

        # Initialization fo hidden and cell states
        torch.nn.init.xavier_normal_(h)
        torch.nn.init.xavier_normal_(c)

        # Each sequence "x" is passed through an embedding layer
        out = self.embedding(text, offsets)
        # Feed LSTMs
        out, (hidden, cell) = self.lstm(out, (h,c))
        out = self.dropout(out)
        # The last hidden state is taken
        out = self.fc(out)

        return out


In [81]:
train_iter = huffPo.get_train()
num_class = len(set([label for (label, text) in train_iter]))
vocab_size = len(vocab)
emsize = 64
model = TextClassificationModel(vocab_size, emsize, num_class).to(device)

In [82]:
import time

def train(dataloader):
    model.train()
    total_acc, total_count = 0, 0
    log_interval = 500
    start_time = time.time()

    for idx, (label, text, offsets) in enumerate(dataloader):
        optimizer.zero_grad()
        predicted_label = model(text, offsets)
        loss = criterion(predicted_label, label)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), 0.1)
        optimizer.step()
        total_acc += (predicted_label.argmax(1) == label).sum().item()
        total_count += label.size(0)
        if idx % log_interval == 0 and idx > 0:
            elapsed = time.time() - start_time
            print('| epoch {:3d} | {:5d}/{:5d} batches '
                  '| accuracy {:8.3f}'.format(epoch, idx, len(dataloader),
                                              total_acc/total_count))
            total_acc, total_count = 0, 0
            start_time = time.time()

def evaluate(dataloader):
    model.eval()
    total_acc, total_count = 0, 0

    with torch.no_grad():
        for idx, (label, text, offsets) in enumerate(dataloader):
            predicted_label = model(text, offsets)
            loss = criterion(predicted_label, label)
            total_acc += (predicted_label.argmax(1) == label).sum().item()
            total_count += label.size(0)
    return total_acc/total_count

In [ ]:
from torch.utils.data.dataset import random_split
from torchtext.data.functional import to_map_style_dataset
# Hyperparameters
EPOCHS = 50 # epoch
LR = 5  # learning rate
BATCH_SIZE = 64 # batch size for training

criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=LR)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, 1.0, gamma=0.1)
total_accu = None
train_iter = huffPo.get_train()
test_iter = huffPo.get_test()
train_dataset = to_map_style_dataset(train_iter)
test_dataset = to_map_style_dataset(test_iter)
num_train = int(len(train_dataset) * 0.95)
split_train_, split_valid_ = \
    random_split(train_dataset, [num_train, len(train_dataset) - num_train])

train_dataloader = DataLoader(split_train_, batch_size=BATCH_SIZE,
                              shuffle=True, collate_fn=collate_batch)
valid_dataloader = DataLoader(split_valid_, batch_size=BATCH_SIZE,
                              shuffle=True, collate_fn=collate_batch)
test_dataloader = DataLoader(test_dataset, batch_size=BATCH_SIZE,
                             shuffle=True, collate_fn=collate_batch)

for epoch in range(1, EPOCHS + 1):
    epoch_start_time = time.time()
    train(train_dataloader)
    accu_val = evaluate(valid_dataloader)
    if total_accu is not None and total_accu > accu_val:
      scheduler.step()
    else:
       total_accu = accu_val
    print('-' * 59)
    print('| end of epoch {:3d} | time: {:5.2f}s | '
          'valid accuracy {:8.3f} '.format(epoch,
                                           time.time() - epoch_start_time,
                                           accu_val))
    print('-' * 59)

| epoch   1 |   500/ 2489 batches | accuracy    0.168
| epoch   1 |  1000/ 2489 batches | accuracy    0.174
| epoch   1 |  1500/ 2489 batches | accuracy    0.197
| epoch   1 |  2000/ 2489 batches | accuracy    0.221
-----------------------------------------------------------
| end of epoch   1 | time:  6.94s | valid accuracy    0.241 
-----------------------------------------------------------
| epoch   2 |   500/ 2489 batches | accuracy    0.251
| epoch   2 |  1000/ 2489 batches | accuracy    0.262
| epoch   2 |  1500/ 2489 batches | accuracy    0.273
| epoch   2 |  2000/ 2489 batches | accuracy    0.285
-----------------------------------------------------------
| end of epoch   2 | time:  6.99s | valid accuracy    0.301 
-----------------------------------------------------------
| epoch   3 |   500/ 2489 batches | accuracy    0.311
| epoch   3 |  1000/ 2489 batches | accuracy    0.319
| epoch   3 |  1500/ 2489 batches | accuracy    0.321
| epoch   3 |  2000/ 2489 batches | accuracy

In [75]:
# print('Test accuracy is: ' + str(evaluate(test_dataloader)))
print(evaluate(test_dataloader))

0.3991313892998616
